#  import modules

In [1]:
import numpy as np
import pandas as pd
import numpy as np
import re

#  import data

In [2]:
df = pd.read_csv("../raw_data/FM_data_21.csv", low_memory = False)

#  data cleaning

###  data exploration

In [3]:
df.head(20)

,Name,Club,Nat,Division,Position,Sec. Position,DoB,Age,Preferred Foot,Right Foot,...,Bal,Kic,Sta,Agi,Wor,Bra,Cmd,Fin,1v1,TRO
0,Kevin De Bruyne,Man City,BEL,English Premier Division ...,"M (RLC), AM (C)",M (RL),28/6/1991 (29 years old),29,Either,Very Strong,...,15,3,17,13,15,12,3,16,1,3
1,Harry Kane,Tottenham,ENG,English Premier Division ...,"AM (C), ST (C)",AM (C),28/7/1993 (27 years old),26,Either,Very Strong,...,13,3,17,12,18,16,2,19,3,3
2,Mohamed Salah,Liverpool,EGY,English Premier Division ...,"AM (RL), ST (C)",AM (L),15/6/1992 (28 years old),28,Left,Weak,...,18,1,14,16,15,11,3,18,3,2
3,Sadio Mané,Liverpool,SEN,English Premier Division ...,"AM (RL), ST (C)",ST (C),10/4/1992 (28 years old),28,Right,Very Strong,...,16,1,14,16,15,11,3,17,2,4
4,Son Heung-Min,Tottenham,KOR,English Premier Division ...,"M/AM (RL), ST (C)","M (RL), AM (R)",8/7/1992 (28 years old),28,Either,Very Strong,...,12,2,16,15,17,6,2,18,2,1
5,Raheem Sterling,Man City,ENG,English Premier Division ...,M/AM (RL),M (RL),8/12/1994 (25 years old),25,Right,Very Strong,...,15,3,17,19,16,11,3,13,3,3
6,Bernardo Silva,Man City,POR,English Premier Division ...,M/AM (RC),M (R),10/8/1994 (25 years old),25,Left,Weak,...,18,3,16,19,20,15,2,16,4,2
7,Roberto Firmino,Liverpool,BRA,English Premier Division ...,"AM (RLC), ST (C)",AM (RL),2/10/1991 (28 years old),28,Right,Very Strong,...,17,3,16,15,18,13,3,13,4,3
8,Virgil van Dijk,Liverpool,NED,English Premier Division ...,D (C),-,8/7/1991 (29 years old),29,Right,Very Strong,...,15,2,14,12,12,15,2,8,1,2
9,Bruno Fernandes,Man Utd,POR,English Premier Division ...,M/AM (C),-,8/9/1994 (25 years old),25,Right,Very Strong,...,14,2,18,15,17,17,2,15,1,1


In [4]:
df.rename(str.lower, axis='columns', inplace = True)
df.rename(str.strip, axis='columns', inplace = True)

In [5]:
df.columns

Index(['name', 'club', 'nat', 'division', 'position', 'sec. position', 'dob',
       'age', 'preferred foot', 'right foot', 'left foot', 'height', 'weight',
       'at lge gls', 'at lge apps', 'wage', 'last trans. fee', 'value',
       'rc injury', 'agg', 'jum', 'pun', 'nat .1', 'vis', 'l th', 'lon', 'otb',
       'tck', 'tec', 'tea', 'cmp', 'fre', 'ref', 'pos', 'pen', 'pas', 'fla',
       'ant', 'cro', 'mar', 'ldr', 'cor', 'cnt', 'det', 'dec', 'hea', 'fir',
       'com', 'acc', 'pac', 'aer', 'str', 'thr', 'han', 'ecc', 'dri', 'bal',
       'kic', 'sta', 'agi', 'wor', 'bra', 'cmd', 'fin', '1v1', 'tro'],
      dtype='object')

###  column definitions

In [6]:
column_definitions = {
    'name': 'Player name',
    'club': 'Current club',
    'nat': 'Nationality',
    'division': 'Club division',
    'position': 'Primary playing position',
    'sec. position': 'Secondary playing position',
    'dob': 'Date of birth',
    'age': 'Player age',
    'preferred foot': 'Preferred foot for playing',
    'right foot': 'Right foot proficiency',
    'left foot': 'Left foot proficiency',
    'height': 'Player height',
    'weight': 'Player weight',
    'at lge gls': 'Goals scored in the league',
    'at lge apps': 'Appearances in the league',
    'wage': 'Players wage',
    'last trans. fee': 'Last transfer fee paid',
    'value': 'Players market value',
    'rc injury': 'Recovery from injury',
    'agg': 'Aggression attribute',
    'jum': 'Jumping attribute',
    'pun': 'Punishment attribute',
    'nat .1': 'Nationality (duplicate)',
    'vis': 'Vision attribute',
    'l th': 'Long throwing attribute',
    'lon': 'Long shots attribute',
    'otb': 'Off the ball attribute',
    'tck': 'Tackling attribute',
    'tec': 'Technique attribute',
    'tea': 'Teamwork attribute',
    'cmp': 'Composure attribute',
    'fre': 'Free-kick taking attribute',
    'ref': 'Reflexes attribute',
    'pos': 'Positioning attribute',
    'pen': 'Penalties attribute',
    'pas': 'Passing attribute',
    'fla': 'Flair attribute',
    'ant': 'Anticipation attribute',
    'cro': 'Crossing attribute',
    'mar': 'Marking attribute',
    'ldr': 'Leadership attribute',
    'cor': 'Corner-taking attribute',
    'cnt': 'Concentration attribute',
    'det': 'Determination attribute',
    'dec': 'Decision-making attribute',
    'hea': 'Heading attribute',
    'fir': 'First touch attribute',
    'com': 'Communication attribute',
    'acc': 'Acceleration attribute',
    'pac': 'Pace attribute',
    'aer': 'Aerial ability attribute',
    'str': 'Strength attribute',
    'thr': 'Throw-in attribute',
    'han': 'Handling attribute',
    'ecc': 'Eccentricity attribute',
    'dri': 'Dribbling attribute',
    'bal': 'Balance attribute',
    'kic': 'Kicking attribute',
    'sta': 'Stamina attribute',
    'agi': 'Agility attribute',
    'wor': 'Work rate attribute',
    'bra': 'Bravery attribute',
    'cmd': 'Command of area attribute',
    'fin': 'Finishing attribute',
    '1v1': 'One-on-one ability attribute',
    'tro': 'Traits attribute'
}

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174574 entries, 0 to 174573
Data columns (total 66 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   name             174574 non-null  object
 1   club             174574 non-null  object
 2   nat              174574 non-null  object
 3   division         174574 non-null  object
 4   position         174574 non-null  object
 5   sec. position    174574 non-null  object
 6   dob              174574 non-null  object
 7   age              174574 non-null  object
 8   preferred foot   174574 non-null  object
 9   right foot       174574 non-null  object
 10  left foot        174574 non-null  object
 11  height           174574 non-null  object
 12  weight           174574 non-null  object
 13  at lge gls       174574 non-null  object
 14  at lge apps      174574 non-null  object
 15  wage             174574 non-null  object
 16  last trans. fee  174574 non-null  object
 17  value     

###  classing positions

In [8]:
# drop sec position column
df.drop(columns = ['sec. position', 'rc injury'], inplace=True)

In [9]:
# copy position column to main_position
df["main_position"] = df.position

# removing the spaces in the string
df.main_position = df.main_position.str.strip().str.replace(',\s*', ',')
 
# making a list out of all the positions
df.main_position = df.main_position.str.split(',')

# peek
df.main_position.head()

/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/2647964038.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df.main_position = df.main_position.str.strip().str.replace(',\s*', ',')


0      [M (RLC), AM (C)]
1       [AM (C), ST (C)]
2      [AM (RL), ST (C)]
3      [AM (RL), ST (C)]
4    [M/AM (RL), ST (C)]
Name: main_position, dtype: object

In [10]:
# dictionary for every position class
football_positions = {
    'goalkeeper': ['GK'],
    'centerback': ['D (C)', 'D (RC)', 'D (LC)', 'D (RLC)', 'D/M (C)'],
    'fullback': ['D/WB/AM (RL)','D/WB/AM (L)','WB/M/AM (L)','WB/M (R)','WB/M/AM (R)','D/WB/M/AM (R)','D/WB/M/AM (L)','D/WB/M (R)','WB (R)','WB/M/AM (RL)','D/WB/M (RL)','D/WB (RL)','WB (L)','D/WB/AM (R)','D/WB (L)','WB (RL)','D/WB (R)','D (RL)', 'D (L)', 'D (R)', 'D (L)', 'D/AM (R)', 'D/AM (L)', 'D/AM (RL)', 'D/WB/M (L)', 'D/M/AM (RL)', 'D/M/AM (L)', 'D/M (R)', 'D/M (L)', 'D/AM (R)', 'WB/M (L)', 'D/M/AM (R)', 'D/WB/M/AM (RL)', 'D/M (R)', 'D/M (L)', 'D/AM (R)', 'WB/AM (L)', 'D/AM (L)', 'D/AM (RL)', 'WB/M (RL)', 'WB/AM (R)', 'D/M (RL)', 'WB/AM (RL)'],
    'midfielder': ['D/M (RC)','D/M/AM (C)','D/AM (C)','D/M (LC)','D/M/AM (LC)','AM (LC)','AM (RLC)','AM (RC)','AM (C)','M (RLC)','M (C)', 'M (LC)', 'M (RC)', 'DM', 'M/AM (C)', 'M/AM (LC)', 'M/AM (RC)', 'M/AM (RLC)'],
    'winger': ['AM (RL)','M (RL)', 'M (R)', 'M (L)', 'M/AM (R)', 'M/AM (L)', 'M/AM (RL)', 'AM (L)', 'AM (R)'],
    'striker': ['ST (C)'],
    "no_position": [""]
}
# making the map with the football positions dictionary
positions_map = {}

for main_position, positions_list in football_positions.items():
    for position in positions_list:
        positions_map[position] = main_position


# peek
positions_map

{'GK': 'goalkeeper',
 'D (C)': 'centerback',
 'D (RC)': 'centerback',
 'D (LC)': 'centerback',
 'D (RLC)': 'centerback',
 'D/M (C)': 'centerback',
 'D/WB/AM (RL)': 'fullback',
 'D/WB/AM (L)': 'fullback',
 'WB/M/AM (L)': 'fullback',
 'WB/M (R)': 'fullback',
 'WB/M/AM (R)': 'fullback',
 'D/WB/M/AM (R)': 'fullback',
 'D/WB/M/AM (L)': 'fullback',
 'D/WB/M (R)': 'fullback',
 'WB (R)': 'fullback',
 'WB/M/AM (RL)': 'fullback',
 'D/WB/M (RL)': 'fullback',
 'D/WB (RL)': 'fullback',
 'WB (L)': 'fullback',
 'D/WB/AM (R)': 'fullback',
 'D/WB (L)': 'fullback',
 'WB (RL)': 'fullback',
 'D/WB (R)': 'fullback',
 'D (RL)': 'fullback',
 'D (L)': 'fullback',
 'D (R)': 'fullback',
 'D/AM (R)': 'fullback',
 'D/AM (L)': 'fullback',
 'D/AM (RL)': 'fullback',
 'D/WB/M (L)': 'fullback',
 'D/M/AM (RL)': 'fullback',
 'D/M/AM (L)': 'fullback',
 'D/M (R)': 'fullback',
 'D/M (L)': 'fullback',
 'WB/M (L)': 'fullback',
 'D/M/AM (R)': 'fullback',
 'D/WB/M/AM (RL)': 'fullback',
 'WB/AM (L)': 'fullback',
 'WB/M (RL)': '

In [11]:
# Function to map positions in a list
def map_positions(position_list):
    return [positions_map[pos] for pos in position_list]

# Apply the mapping function to the 'position' column
df.main_position = df.main_position.apply(map_positions)

# peek
df.main_position.head()

0    [midfielder, midfielder]
1       [midfielder, striker]
2           [winger, striker]
3           [winger, striker]
4           [winger, striker]
Name: main_position, dtype: object

In [12]:
# Returns "striker" if it's in the list, otherwise returns the first position.
def select_main_position(positions):
    if positions == ["midfielder", "striker"]:
        return "striker"
    return positions[0]

# Applies the select_main_position function to the 'main_position' column in the DataFrame.
df.main_position = df.main_position.apply(select_main_position)

In [13]:
# peek
df.main_position.head(20)

0     midfielder
1        striker
2         winger
3         winger
4         winger
5         winger
6     midfielder
7        striker
8     centerback
9     midfielder
10    centerback
11    midfielder
12       striker
13    midfielder
14    centerback
15    midfielder
16      fullback
17    goalkeeper
18    midfielder
19    centerback
Name: main_position, dtype: object

###  changing the types of the features

#### defining regex 

In [14]:
def extract_numbers(input_string):
    # Use regular expression to extract numbers
    numbers = re.findall(r'\d+', input_string)
    
    # Join the numbers into a single string or convert to a list depending on your needs
    result = ''.join(numbers)  # If you want a single string
    # result = [int(num) for num in numbers]  # If you want a list of integers
    
    return result

def extract_date(input_string):
    # Use regular expression to extract the date
    date_match = re.search(r'\b\d{1,2}/\d{1,2}/\d{4}\b', input_string)

    if date_match:
        return date_match.group()
    else:
        return None

def extract_amount(input_string):
    # Use regular expression to extract numeric values with optional K or M suffix
    match = re.search(r'(?i)([0-9,.]+)([km]?)', input_string)
    
    if match:
        value_str, suffix = match.groups()
        value = float(value_str.replace(',', '.'))

        if suffix.lower() == 'k':
            value *= 1e3
        elif suffix.lower() == 'm':
            value *= 1e6

        return value
    else:
        return None

#### converting columntypes

In [15]:
# applying regex to features with 'dirty' numbers

df.dob = df.dob.apply(extract_date)
df.height = df.height.apply(extract_numbers)
df.weight = df.weight.apply(extract_numbers)
df.wage = df.wage.apply(extract_numbers)
df['last trans. fee'] = df['last trans. fee'].apply(extract_amount)
df.value = df.value.apply(extract_amount)

In [16]:
numerical_columns = ['age', 'height', 'weight', 'at lge gls', 'at lge apps', 'wage', 'last trans. fee', 'value',
                      'agg', 'jum', 'pun', 'nat .1', 'vis', 'l th', 'lon', 'otb', 'tck', 'tec', 'tea',
                      'cmp', 'fre', 'ref', 'pos', 'pen', 'pas', 'fla', 'ant', 'cro', 'mar', 'ldr', 'cor', 'cnt', 'det',
                      'dec', 'hea', 'fir', 'com', 'acc', 'pac', 'aer', 'str', 'thr', 'han', 'ecc', 'dri', 'bal', 'kic',
                      'sta', 'agi', 'wor', 'bra', 'cmd', 'fin', '1v1', 'tro']


In [17]:
df['last trans. fee']

0         60000000.0
1                NaN
2         37500000.0
3         40000000.0
4         24000000.0
             ...    
174569           NaN
174570           NaN
174571           NaN
174572           NaN
174573           NaN
Name: last trans. fee, Length: 174574, dtype: float64

In [18]:
# Convert numerical columns to float32, replacing non-convertible values with NaN
df[numerical_columns] = df[numerical_columns].apply(pd.to_numeric, errors='coerce').astype('float32')

# Convert 'dob' column to datetime
df['dob'] = pd.to_datetime(df['dob'], errors='coerce')

/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/273099830.py:5: UserWarning: Parsing '23/10/1993' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['dob'] = pd.to_datetime(df['dob'], errors='coerce')
/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/273099830.py:5: UserWarning: Parsing '24/10/1990' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['dob'] = pd.to_datetime(df['dob'], errors='coerce')
/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/273099830.py:5: UserWarning: Parsing '31/10/1997' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['dob'] = pd.to_datetime(df['dob'], errors='coerce')
/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/273099830.py:5: UserWarning: Parsing '14/12/1996' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=Tr

/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/273099830.py:5: UserWarning: Parsing '31/12/2001' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['dob'] = pd.to_datetime(df['dob'], errors='coerce')
/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/273099830.py:5: UserWarning: Parsing '21/10/1997' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['dob'] = pd.to_datetime(df['dob'], errors='coerce')
/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/273099830.py:5: UserWarning: Parsing '27/11/1988' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['dob'] = pd.to_datetime(df['dob'], errors='coerce')
/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/273099830.py:5: UserWarning: Parsing '21/11/1999' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=Tr

/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/273099830.py:5: UserWarning: Parsing '30/11/1998' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['dob'] = pd.to_datetime(df['dob'], errors='coerce')
/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/273099830.py:5: UserWarning: Parsing '29/11/1986' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['dob'] = pd.to_datetime(df['dob'], errors='coerce')
/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/273099830.py:5: UserWarning: Parsing '13/11/2001' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['dob'] = pd.to_datetime(df['dob'], errors='coerce')
/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/273099830.py:5: UserWarning: Parsing '25/11/1985' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=Tr

/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/273099830.py:5: UserWarning: Parsing '27/12/1994' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['dob'] = pd.to_datetime(df['dob'], errors='coerce')
/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/273099830.py:5: UserWarning: Parsing '29/12/1983' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['dob'] = pd.to_datetime(df['dob'], errors='coerce')
/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/273099830.py:5: UserWarning: Parsing '19/10/1999' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['dob'] = pd.to_datetime(df['dob'], errors='coerce')
/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/273099830.py:5: UserWarning: Parsing '14/11/1981' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=Tr

/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/273099830.py:5: UserWarning: Parsing '20/10/1984' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['dob'] = pd.to_datetime(df['dob'], errors='coerce')
/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/273099830.py:5: UserWarning: Parsing '28/12/1983' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['dob'] = pd.to_datetime(df['dob'], errors='coerce')
/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/273099830.py:5: UserWarning: Parsing '19/11/1980' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['dob'] = pd.to_datetime(df['dob'], errors='coerce')
/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/273099830.py:5: UserWarning: Parsing '22/10/1989' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=Tr

/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/273099830.py:5: UserWarning: Parsing '15/11/1979' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['dob'] = pd.to_datetime(df['dob'], errors='coerce')
/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/273099830.py:5: UserWarning: Parsing '15/10/1973' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['dob'] = pd.to_datetime(df['dob'], errors='coerce')
/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/273099830.py:5: UserWarning: Parsing '16/12/1980' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['dob'] = pd.to_datetime(df['dob'], errors='coerce')
/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/273099830.py:5: UserWarning: Parsing '20/12/1973' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=Tr

/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/273099830.py:5: UserWarning: Parsing '18/10/1982' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['dob'] = pd.to_datetime(df['dob'], errors='coerce')
/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/273099830.py:5: UserWarning: Parsing '24/12/1989' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['dob'] = pd.to_datetime(df['dob'], errors='coerce')
/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/273099830.py:5: UserWarning: Parsing '22/12/1989' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['dob'] = pd.to_datetime(df['dob'], errors='coerce')
/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/273099830.py:5: UserWarning: Parsing '25/11/1981' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=Tr

/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/273099830.py:5: UserWarning: Parsing '18/12/1979' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['dob'] = pd.to_datetime(df['dob'], errors='coerce')
/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/273099830.py:5: UserWarning: Parsing '17/11/1982' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['dob'] = pd.to_datetime(df['dob'], errors='coerce')
/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/273099830.py:5: UserWarning: Parsing '31/10/1984' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['dob'] = pd.to_datetime(df['dob'], errors='coerce')
/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/273099830.py:5: UserWarning: Parsing '25/11/1984' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=Tr

/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/273099830.py:5: UserWarning: Parsing '26/11/1979' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['dob'] = pd.to_datetime(df['dob'], errors='coerce')
/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/273099830.py:5: UserWarning: Parsing '22/11/1982' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['dob'] = pd.to_datetime(df['dob'], errors='coerce')
/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/273099830.py:5: UserWarning: Parsing '22/10/1985' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['dob'] = pd.to_datetime(df['dob'], errors='coerce')
/var/folders/pv/hj9fs4fx3fn0k6t5l6qmlh6m0000gn/T/ipykernel_69381/273099830.py:5: UserWarning: Parsing '17/12/1982' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=Tr

In [19]:
# check
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174574 entries, 0 to 174573
Data columns (total 66 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   name             174574 non-null  object        
 1   club             174574 non-null  object        
 2   nat              174574 non-null  object        
 3   division         174574 non-null  object        
 4   position         174574 non-null  object        
 5   dob              174324 non-null  datetime64[ns]
 6   age              174324 non-null  float32       
 7   preferred foot   174574 non-null  object        
 8   right foot       174574 non-null  object        
 9   left foot        174574 non-null  object        
 10  height           174324 non-null  float32       
 11  weight           174324 non-null  float32       
 12  at lge gls       86208 non-null   float32       
 13  at lge apps      121188 non-null  float32       
 14  wage             127

#### onehotencode main position

In [20]:
df.head()

,name,club,nat,division,position,dob,age,preferred foot,right foot,left foot,...,kic,sta,agi,wor,bra,cmd,fin,1v1,tro,main_position
0,Kevin De Bruyne,Man City,BEL,English Premier Division ...,"M (RLC), AM (C)",1991-06-28,29.0,Either,Very Strong,Strong,...,3.0,17.0,13.0,15.0,12.0,3.0,16.0,1.0,3.0,midfielder
1,Harry Kane,Tottenham,ENG,English Premier Division ...,"AM (C), ST (C)",1993-07-28,26.0,Either,Very Strong,Very Strong,...,3.0,17.0,12.0,18.0,16.0,2.0,19.0,3.0,3.0,striker
2,Mohamed Salah,Liverpool,EGY,English Premier Division ...,"AM (RL), ST (C)",1992-06-15,28.0,Left,Weak,Very Strong,...,1.0,14.0,16.0,15.0,11.0,3.0,18.0,3.0,2.0,winger
3,Sadio Mané,Liverpool,SEN,English Premier Division ...,"AM (RL), ST (C)",1992-10-04,28.0,Right,Very Strong,Reasonable,...,1.0,14.0,16.0,15.0,11.0,3.0,17.0,2.0,4.0,winger
4,Son Heung-Min,Tottenham,KOR,English Premier Division ...,"M/AM (RL), ST (C)",1992-08-07,28.0,Either,Very Strong,Very Strong,...,2.0,16.0,15.0,17.0,6.0,2.0,18.0,2.0,1.0,winger


In [22]:
df = pd.get_dummies(df, columns=['main_position'], prefix="", prefix_sep="")

In [30]:
# peek
df.sample(20)

,name,club,nat,division,position,dob,age,preferred foot,right foot,left foot,...,fin,1v1,tro,centerback,fullback,goalkeeper,midfielder,no_position,striker,winger
31318,Joseph Rabbetts,Aldershot,ENG,Vanarama National League ...,D/WB/M (L),2002-07-17,18.0,Left Only,Very Weak,Very Strong,...,3.0,2.0,2.0,0,1,0,0,0,0,0
160875,Jordan Wilson,York9 FC,CAN,Canadian Premier League,"D (RC), DM, M (C)",1991-10-31,28.0,Right,Very Strong,Reasonable,...,5.0,1.0,1.0,1,0,0,0,0,0,0
31435,Keivan Ghaedamini,KFUM Oslo,NOR,Norwegian First Division ...,D/WB (RL),1996-09-03,24.0,Either,Very Strong,Strong,...,6.0,3.0,1.0,0,1,0,0,0,0,0
40871,Oliver Nedanovski,Åryds IK,SWE,Swedish Third Division Southeast Götaland ...,"D (C), DM, M (C)",1991-05-13,29.0,Left,Reasonable,Very Strong,...,10.0,3.0,3.0,1,0,0,0,0,0,0
58173,Númi Már Atlason,Berserkir,ISL,Icelandic Lower Division ...,ST (C),1998-05-02,22.0,Right Only,Very Strong,Weak,...,10.0,3.0,2.0,0,0,0,0,0,1,0
43451,Ante Barać,Sloga Mravince,CRO,Croatian Third League South ...,D (RLC),1995-03-28,25.0,Right,Very Strong,Weak,...,5.0,2.0,3.0,1,0,0,0,0,0,0
122028,Thiago Arroyo,Barracas Central,ARG,Argentine Second Division,ST (C),2003-11-26,16.0,Right Only,Very Strong,Weak,...,14.0,3.0,4.0,0,0,0,0,0,1,0
82698,Mamadou Diatta,Pau FC,SEN,Ligue 2 BKT ...,M (C),2001-05-04,19.0,Right,Very Strong,Reasonable,...,10.0,1.0,2.0,0,0,0,1,0,0,0
22567,Cristian Cocan,CSO Cugir,ROU,Romanian Third League Group 9 ...,"M/AM (C), ST (C)",1995-03-12,24.0,Right Only,Very Strong,Weak,...,6.0,1.0,2.0,0,0,0,0,0,1,0
168791,Palauni Tapusoa,Utulei,ASA,American Samoan Senior League Division One,"AM (R), ST (C)",1994-01-02,26.0,Right Only,Very Strong,Weak,...,10.0,1.0,4.0,0,0,0,0,0,0,1


#### quantify division

In [28]:
df.division.unique()

array([' English Premier Division                           ',
       ' Sky Bet Championship                               ',
       ' Scottish Premiership                               ', ...,
       ' MLS Next - Atlantic Division                  ',
       ' MLS Next - Frontier Division                  ',
       ' MLS Next - Southwest Division                 '], dtype=object)

##  data cleaning